In [1]:
# to generate result sheet autoly
import pandas as pd 
import numpy as np 
import scipy
# import xlrd 
import sklearn

from Gibbs_model_probit import Gibbs_sampling
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from utils import baseline_lr,baseline_esnet,baseline_justmean
from utils import baseline_LogitElsnet,baseline_justmode,baseline_random,baseline_LogitLR,baseline_RanForest,baseline_Gibbs_zhe,baseline_SVM
from utils import baseline_LogitElsnet_leaveone,baseline_LogitLR_leaveone,baseline_SVM_leaveone
from utils import logis_regression_5,kernel_svm_regression_5,random_forest_regression_5,gpr_regression_5,logis_lasso_regression_5
from sklearn.model_selection import KFold
from scipy.stats import binom 
from scipy.stats import norm
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from tqdm import trange
from Gibbd_zhe import GibbsSampling3
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import LeaveOneOut
import time
from sklearn.preprocessing import StandardScaler
# from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern
from sklearn.linear_model import LogisticRegression

In [2]:
# data_loading 
np.random.seed(123)

# data_table = pd.read_csv('../data/processed/all_feature_p1_lip_total_single.csv')
# data_table_cont = pd.read_csv('../data/processed/all_feature_p1_spicie_cont_single.csv')
# data_table_binary = pd.read_csv('../data/processed/all_feature_p1_spicie_binary_single.csv')
# data_table_binary_standard = pd.read_csv('../data/processed/all_feature_p1_spicie_binary_standard_single.csv')
data_table_cont = pd.read_csv('../data/processed/all_feature_p2_spicie_cont_single.csv')
data_table_binary = pd.read_csv('../data/processed/all_feature_p2_spicie_binary_single.csv')



In [3]:
all_feature_cont = data_table_cont.iloc[:,1:-3]
all_feature_binary = data_table_binary.iloc[:,1:-3]
all_feature_binary

,lip_signature,lip_P2_Sph_Sph(d16:1)_high,lip_P2_Sph_Sph(d16:1)_low,lip_P2_Sph_Sph(d18:1)_high,lip_P2_Sph_Sph(d18:1)_low,lip_P2_Sph_Sph(d18:2)_high,lip_P2_Sph_Sph(d18:2)_low,lip_P2_Cer_Cer(d16:1/16:0)_high,lip_P2_Cer_Cer(d16:1/16:0)_low,lip_P2_Cer_Cer(d16:1/18:0)_high,...,lip_P2_Acylcarnitine_AC(17:0) (b)_low,lip_P2_Acylcarnitine_AC(18:0)_high,lip_P2_Acylcarnitine_AC(18:0)_low,lip_P2_Acylcarnitine_AC(18:1)_high,lip_P2_Acylcarnitine_AC(18:1)_low,lip_P2_Acylcarnitine_AC(18:2)_high,lip_P2_Acylcarnitine_AC(18:2)_low,gene_P2_AR_Amp,gene_P2_RB1_Del,gene_P2_TP53_SNP
0,0,1,0,0,1,0,1,1,0,1,...,0,1,0,1,0,1,0,0,0,0
1,0,1,0,1,0,1,0,0,1,0,...,0,0,1,1,0,1,0,1,0,1
2,1,1,0,1,0,0,1,0,1,0,...,1,0,1,0,1,1,0,0,0,0
3,0,0,1,0,1,0,1,1,0,1,...,0,1,0,1,0,1,0,0,1,1
4,1,0,1,0,1,1,0,1,0,1,...,0,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1,1,0,1,0,0,1,1,0,1,...,0,0,1,0,1,0,1,0,0,0
140,0,1,0,1,0,0,1,1,0,0,...,1,0,1,0,1,0,1,0,0,1
141,1,1,0,0,1,1,0,0,1,1,...,1,0,1,0,1,0,1,0,0,0
142,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,1


In [4]:
# P1(mHSPC) testtosterone suppression failure: CER(ceramide), DG(diacylglycerol), TG(triaclyglycerol)
# P2(mCRPC) overall survival:  CER(ceramide), Sph(Sphingosine), Acylcarnitine(acylcarnitine)
def feature_filter(df):
    all_feature = list(df)
    feature_dict = {}
    feature_dict['gene'] = [item for item in all_feature if 'gene_' in item]
    # feature_dict['TG'] = [item for item in all_feature if 'lip_P1' in item and 'lip_P1_TG' not in item] # remove the TG lip
    # lip_feature_P2 = [item for item in all_feature if 'lip_P2' in item and '_Cer' not in item]
    # lip_feature_all =  [item for item in all_feature if 'lip_' in item and 'P1_Cer' not in item]
    
    feature_dict['TG'] = [item for item in all_feature if 'lip_P1_TG' in item]

    feature_dict['Cer'] = [item for item in all_feature if '_Cer' in item]
    feature_dict['DG'] = [item for item in all_feature if 'lip_P1_DG' in item]

    feature_dict['Acy'] = [item for item in all_feature if '_Acylcarnitine' in item]
    feature_dict['Sph'] = [item for item in all_feature if '_Sph' in item]
    feature_dict['sig'] = [item for item in all_feature if 'signature' in item]

    return feature_dict

In [5]:
# gene_feature,lip_feature_P1,lip_feature_P2,lip_feature_all,\
#     lip_feature_TG,lip_feature_Cer,lip_feature_DG,\
#         lip_feature_Acylcarnitine,lip_feature_Sph,lip_feature_sig = feature_filter(data_table_binary)

# data_table = data_table_cont
data_table = data_table_binary
# data_table = data_table_binary_standard


feature_dict = feature_filter(data_table)
one_feature_P1 = [['TG'],['Cer'],['DG']]
one_feature_P2 = [['Cer'],['Acy'],['Sph']]

two_feature_P1 = [['TG','Cer'],['TG','DG'],['DG','Cer']]
two_feature_P2 = [['Acy','Cer'],['Sph','Cer'],['Sph','Acy']]

three_feature_P1 = [['TG','Cer','DG']]
three_feature_P2 = [['Cer','Acy','Sph']]


In [13]:
index = ['logistic reg','logistic reg-lasso','linear-kernel svm','RBF-kernel svm','RBF-kernel GPR','matern-kernel GPR' ]
result_table = pd.DataFrame(index = index)



# target ='1= death; 0=alive'
target ='gap_surv_time_class'
sub_target = '_early_death'
# sub_target = '_long_survive'

# target = 'ADT_if_fail'

Y = data_table_cont[target].values

if target == 'gap_surv_time_class':
    # for this target, Y=0 means early death, Y=1 means long survival, Y=2 for the middle value
    if sub_target == '_early_death':
        Y = (Y==0)*1 # early death predict
        target = target + sub_target
    elif sub_target == '_long_survive':
        Y = (Y==1)*1 # long survive predict
        target = target + sub_target


In [21]:
Y.sum()

45

In [7]:
# single test gene:


kernel_white = DotProduct() + WhiteKernel()
kernel_RBF = RBF()
kernel_matern = Matern()


# target = 'ADT_if_fail'#'1= death; 0=alive'#'gap_surv_time_class' 'ADT_if_fail'
# Y = data_table_cont[target].values
# Y = (Y==0)*1


X = data_table[feature_dict['gene']].values
log_loss = logis_regression_5(X,Y)
log_lasso_loss = logis_lasso_regression_5(X,Y,0.1)
svm_loss_rbf = kernel_svm_regression_5(X,Y,'rbf')
svm_loss_linear = kernel_svm_regression_5(X,Y,'linear')

# gpr_white_loss = gpr_regression_5(X,Y,kernel_white)
gpr_RBF_loss = gpr_regression_5(X,Y,kernel_RBF)
gpr_matern_loss = gpr_regression_5(X,Y,kernel_matern)

loss = [log_loss,log_lasso_loss,svm_loss_linear,svm_loss_rbf,gpr_RBF_loss,gpr_matern_loss]

result_table['gene'] = loss

G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\gaussian_process\_gpr.py:506: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [9]:
# target = '1= death; 0=alive'#'gap_surv_time_class' 'ADT_if_fail'
# target = '1= death; 0=alive'#'gap_surv_time_class' 'ADT_if_fail'
# Y = data_table_cont[target].values

def all_model(data_table,result_table,features,feature_dict,Y,add_gene=False):


    for fes in features:
        
        feature_name = '_'.join(fes)
        feature_list = [data_table[feature_dict[item]].values for item in fes]

        if add_gene:
            feature_name = 'gene_'+feature_name
            feature_list.append(data_table[feature_dict['gene']].values)

        X = np.concatenate(feature_list,1)
        log_loss = logis_regression_5(X,Y)
        log_lasso_loss = logis_lasso_regression_5(X,Y,0.1)
        svm_loss_rbf = kernel_svm_regression_5(X,Y,'rbf')
        svm_loss_linear = kernel_svm_regression_5(X,Y,'linear')

        gpr_RBF_loss = gpr_regression_5(X,Y,kernel_RBF)
        gpr_matern_loss = gpr_regression_5(X,Y,kernel_matern)

        loss = [log_loss,log_lasso_loss,svm_loss_linear,svm_loss_rbf,gpr_RBF_loss,gpr_matern_loss]
        
        result_table[feature_name] = loss

all_model(data_table=data_table,\
        result_table=result_table,\
        features=one_feature_P2,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=False)
all_model(data_table=data_table,\
        result_table=result_table,\
        features=one_feature_P2,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=True)

all_model(data_table=data_table,\
        result_table=result_table,\
        features=two_feature_P2,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=False)
all_model(data_table=data_table,\
        result_table=result_table,\
        features=two_feature_P2,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=True)   
          
all_model(data_table=data_table,\
        result_table=result_table,\
        features=three_feature_P2,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=False)
all_model(data_table=data_table,\
        result_table=result_table,\
        features=three_feature_P2,\
        feature_dict=feature_dict,Y=Y,\
        add_gene=True)


G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
G:\anaconda\envs\pt1.9\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 

In [10]:
result_table

,gene,Cer,Acy,Sph,gene_Cer,gene_Acy,gene_Sph,Acy_Cer,Sph_Cer,Sph_Acy,gene_Acy_Cer,gene_Sph_Cer,gene_Sph_Acy,Cer_Acy_Sph,gene_Cer_Acy_Sph
logistic reg,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005
logistic reg-lasso,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005
linear-kernel svm,0.833005,0.715764,0.833005,0.833005,0.729803,0.839901,0.833005,0.742857,0.722414,0.833005,0.750246,0.729064,0.833005,0.791379,0.763547
RBF-kernel svm,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005,0.833005
RBF-kernel GPR,0.833005,0.735714,0.166995,0.166995,0.749754,0.166995,0.166995,0.749754,0.735714,0.166995,0.749507,0.771182,0.166995,0.735714,0.770690
matern-kernel GPR,0.833005,0.763054,0.166995,0.166995,0.763300,0.166995,0.166995,0.777340,0.791626,0.166995,0.798522,0.798522,0.166995,0.805172,0.798768


In [11]:
file_name = './result_log/P2_pred_'+target+'.csv'
result_table.to_csv(file_name)

In [18]:
# weight analysis 

def logis_regression_weight_analysis(X,Y,l1_ratio=0.5):
    
    kf = KFold(n_splits=5)
    kf.get_n_splits(X)

    clf_list = []

    best_index = 0
    best_score = 0
    loss = []
    
    for i,index in enumerate (kf.split(X)):
        train_index, test_index = index
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        clf = LogisticRegression(random_state=0,penalty='elasticnet',solver='saga',l1_ratio=l1_ratio
).fit(X_train, y_train)
        clf_list.append(clf)

        test_score = clf.score(X_test,y_test)
        # print(test_score)
        loss.append(test_score)

        if test_score>best_score:
            best_index = i
            best_score = test_score
    

    return np.array(loss).mean(),clf_list[best_index]


def get_sparse_name(clf,feature_name,top=30):
    weight = clf.coef_.squeeze()
    
    weight_table = pd.DataFrame()
    weight_table['feature_name'] = feature_name
    weight_table['feature_weight'] = clf.coef_.squeeze()
    weight_table_de  = weight_table.sort_values('feature_weight').iloc[0:top]
    weight_table_in  = weight_table.sort_values('feature_weight',ascending=False).iloc[0:top]

    return weight_table_de,weight_table_in


fes = ['Sph','Cer','gene']# optimal feature groups for early-death prediction
# fes = ['TG','Cer','DG','gene']# optimal feature groups for long-survive prediction

feature_name = [name for item in fes for name in feature_dict[item]]

feature_list = [data_table[feature_dict[item]].values for item in fes]
X = np.concatenate(feature_list,1)

loss, clf=logis_regression_weight_analysis(X,Y,0.1)
print('loss',loss)
weight_table_de,weight_table_in = get_sparse_name(clf,feature_name,top=30)


loss 0.6810344827586207


F:\anaconda\conda\envs\pt\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\anaconda\conda\envs\pt\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\anaconda\conda\envs\pt\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\anaconda\conda\envs\pt\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\anaconda\conda\envs\pt\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_

In [19]:
# def table_process():

def str_clean(string):
    split_list = string.split('_')
    new_list = [split_list[0]]+split_list[2:]
    return '_'.join(new_list)

def effect_ratio(table):
    table['relative_effects_ratio'] = table['feature_weight']/table['feature_weight'].sum()

def table_process(table):

    table[['feature_name']] = table[['feature_name']].applymap(str_clean)
    effect_ratio(table)

table_process(weight_table_de)
table_process(weight_table_in)

file_name_de = './result_log/P2_weight_'+target+'_de_binary'+'.csv'
file_name_in = './result_log/P2_weight_'+target+'_in_binary'+'.csv'
weight_table_de.to_csv(file_name_de)
weight_table_in.to_csv(file_name_in)


In [20]:
weight_table_in

,feature_name,feature_weight,relative_effects_ratio
129,gene_RB1_Del,2.207973,0.153691
39,lip_Cer_Cer(d18:1/17:0)_low,0.699267,0.048674
98,lip_Cer_Cer(d20:1/23:0)_high,0.680954,0.047399
1,lip_Sph_Sph(d16:1)_low,0.592132,0.041217
54,lip_Cer_Cer(d18:1/24:1)_high,0.536894,0.037372
32,lip_Cer_Cer(d17:1/24:1)_high,0.529827,0.036880
91,lip_Cer_Cer(d19:1/24:0)_low,0.527095,0.036690
23,lip_Cer_Cer(d17:1/18:0)_low,0.511378,0.035596
96,lip_Cer_Cer(d20:1/22:0)_high,0.504183,0.035095
116,lip_Cer_Cer(m18:1/18:0)_high,0.499541,0.034772


In [21]:
weight_table_de

,feature_name,feature_weight,relative_effects_ratio
38,lip_Cer_Cer(d18:1/17:0)_high,-0.741108,0.054494
99,lip_Cer_Cer(d20:1/23:0)_low,-0.722801,0.053148
0,lip_Sph_Sph(d16:1)_high,-0.633949,0.046615
55,lip_Cer_Cer(d18:1/24:1)_low,-0.578739,0.042555
33,lip_Cer_Cer(d17:1/24:1)_low,-0.571670,0.042035
90,lip_Cer_Cer(d19:1/24:0)_high,-0.568692,0.041816
22,lip_Cer_Cer(d17:1/18:0)_high,-0.553157,0.040674
97,lip_Cer_Cer(d20:1/22:0)_low,-0.545965,0.040145
117,lip_Cer_Cer(m18:1/18:0)_low,-0.541400,0.039809
10,lip_Cer_Cer(d16:1/20:0)_high,-0.522836,0.038444
